In [65]:
import psycopg2
import os
from dotenv import load_dotenv
from psycopg2.extras import RealDictCursor
from datetime import datetime, timedelta
from app import convert_date_to_week_format, get_db_connection, process_shipping_plans_carry_over

# .env 파일에서 환경 변수 로드
load_dotenv()

True

In [56]:
supplier_filter = 'MOKA'
to_site_filter = 'ECHK'
week_from = convert_date_to_week_format(datetime.now())
week_to = convert_date_to_week_format(datetime.now() + timedelta(weeks=4))

In [57]:
sp_conditions = ["is_deleted = FALSE"]
sp_params = []

In [58]:
if supplier_filter:
    sp_conditions.append("from_site = %s")
    sp_params.append(supplier_filter)
if to_site_filter:
    sp_conditions.append("to_site ILIKE %s")
    sp_params.append(f"%{to_site_filter}%")
if week_from and week_to:
    sp_conditions.append("""(
        (shipping_week >= %s AND shipping_week <= %s AND is_finished = FALSE)
        OR (shipping_week < %s AND is_finished = FALSE)
    )""")
    sp_params.extend([week_from, week_to, week_from])
else:
    sp_conditions.append("is_finished = FALSE")
sp_where_clause = " AND ".join(sp_conditions)

In [59]:
conn = get_db_connection()
cursor = conn.cursor()
cursor.execute(f'''
            SELECT * FROM shipping_plans
            WHERE {sp_where_clause}
            ORDER BY id DESC
        ''', sp_params)
plans = cursor.fetchall()

In [60]:
po_conditions = ["status = 'Active'"]
po_params = []
if supplier_filter:
    po_conditions.append("from_site = %s")
    po_params.append(supplier_filter)
if to_site_filter:
    po_conditions.append("to_site ILIKE %s")
    po_params.append(f"%{to_site_filter}%")
week_from_date = week_from[:10] if week_from else ''
week_to_date = week_to[:10] if week_to else ''
if week_from and week_to:
    po_conditions.append("""(
        (rsd >= %s AND rsd <= %s AND is_finished = FALSE)
        OR (rsd < %s AND is_finished = FALSE)
    )""")
    po_params.extend([week_from_date, week_to_date, week_from_date])
elif week_from:
    po_conditions.append("""(
        rsd >= %s AND is_finished = FALSE
        OR (rsd < %s AND is_finished = FALSE)
    )""")
    po_params.extend([week_from_date, week_from_date])
elif week_to:
    po_conditions.append("rsd <= %s AND is_finished = FALSE")
    po_params.append(week_to_date)
else:
    po_conditions.append("is_finished = FALSE")
po_where_clause = " AND ".join(po_conditions)
cursor.execute(f'''
    SELECT po_number, from_site, to_site, model, po_qty, rsd FROM purchase_orders
    WHERE {po_where_clause}
    ORDER BY id DESC
''', po_params)
po_rows = cursor.fetchall()

In [66]:
default_weekname = convert_date_to_week_format(datetime.now())

In [68]:
process_shipping_plans_carry_over(plans, default_weekname)

[RealDictRow([('id', 9),
              ('from_site', 'MOKA'),
              ('to_site', 'ECHK'),
              ('model_name', '34G630A-B.AHKQ'),
              ('shipping_week', '2025-10-27(W44)'),
              ('shipping_quantity', 140),
              ('shipped_quantity', 0),
              ('is_finished', False),
              ('remark', ''),
              ('created_at',
               datetime.datetime(2025, 10, 22, 17, 42, 38, 978894, tzinfo=datetime.timezone.utc)),
              ('updated_at',
               datetime.datetime(2025, 10, 22, 17, 42, 38, 978894, tzinfo=datetime.timezone.utc)),
              ('is_deleted', False),
              ('original_shipping_week', '2025-10-20(W43)')])]

In [62]:
po_rows

[RealDictRow([('po_number', 'JGMZ25025676'),
              ('from_site', 'MOKA'),
              ('to_site', 'ECHK'),
              ('model', '34G630A-B.AHKQ'),
              ('po_qty', 9),
              ('rsd', datetime.date(2025, 10, 26))]),
 RealDictRow([('po_number', 'JGMZ25025042'),
              ('from_site', 'MOKA'),
              ('to_site', 'ECHK'),
              ('model', '34G630A-B.AHKQ'),
              ('po_qty', 131),
              ('rsd', datetime.date(2025, 10, 26))])]

In [63]:
po_map = {}
for po in po_rows:
    weekname = convert_date_to_week_format(po['rsd']) if po['rsd'] else ''
    key = (po['from_site'], po['to_site'], po['model'], weekname)
    po_map[key] = po

In [64]:
po_map

{('MOKA',
  'ECHK',
  '34G630A-B.AHKQ',
  '2025-10-20(W43)'): RealDictRow([('po_number', 'JGMZ25025042'),
              ('from_site', 'MOKA'),
              ('to_site', 'ECHK'),
              ('model', '34G630A-B.AHKQ'),
              ('po_qty', 131),
              ('rsd', datetime.date(2025, 10, 26))])}